In [1]:
import tensorflow as tf

# 1. 交叉熵

In [2]:
labels = [[0, 0, 1],[0, 1, 0]] # label是标签值
logits = [[2, 0.5, 6], [0.1, 0, 3]] # logits是网络输出值

# 对输出值计算两次softmax
logits_scaled = tf.nn.softmax(logits)
logits_scaled2 = tf.nn.softmax(logits_scaled)

# result1计算的是labels和logits的softmax交叉熵
result1 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
result2 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits_scaled)
result3 = -tf.reduce_sum(labels*tf.log(logits_scaled),1)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



__打印不同处理方式后得到的结果：__

original logits= [[2, 0.5, 6], [0.1, 0, 3]]

__经过第一次softmax处理之后，数组中最大的数值会接近于1，其余接近0,总和为1__

logits_scaled= [[0.01791432 0.00399722 0.97808844], [0.04980332 0.04506391 0.90513283]]

__再一次softmax后，概率会变化，因为上一次softmax使得所有数值在(0,1)之间分布，结果之间的差值<1, 所以本次输出会让概率变得接近__

logits_scaled2= [[0.21747023 0.21446465 0.56806517], [0.2300214  0.22893383 0.5410447 ]]

__计算分类结果和标签的交叉熵：__

__结果中第一类和标签相符，第二类和标签不符，所以第一类交叉熵较小，第二类较大__

result1= [0.02215516 3.0996735 ] 

__比较result2和result1，说明使用tf.nn.softmax_cross_entropy_with_logits()时，logits无需进行softmax转换；若传入softmax之后的logits，则相当于进行2次softmax转换__

result2= [0.56551915 1.4743223 ] 

__若在传入之前logits已经进行了softmax转换，则不能直接传入softmax_cross_entropy_with_logits(),否则相当于2次转换；而是自行构建loss函数以实现功能__

result3= [0.02215518 3.0996735 ]

In [7]:
with tf.Session() as sess:
    print ("original logits=", logits, "\n")
    print ("logits_scaled=",sess.run(logits_scaled), "\n")    
    print ("logits_scaled2=",sess.run(logits_scaled2), "\n") #经过第二次的softmax后，分布概率会有变化
    

    print ("result1=",sess.run(result1),"\n")#正确的方式
    print ("result2=",sess.run(result2),"\n")#如果将softmax变换完的值放进去会，就相当于算第二次softmax的loss，所以会出错
    print ("result3=",sess.run(result3))

original logits= [[2, 0.5, 6], [0.1, 0, 3]] 

logits_scaled= [[0.01791432 0.00399722 0.97808844]
 [0.04980332 0.04506391 0.90513283]] 

logits_scaled2= [[0.21747023 0.21446465 0.56806517]
 [0.2300214  0.22893383 0.5410447 ]] 

result1= [0.02215516 3.0996735 ] 

result2= [0.56551915 1.4743223 ] 

result3= [0.02215518 3.0996735 ]


# 2. one-hot编码

比较非标准one-hot（总和为1，但是数组中的值不是1或0）为标签的数据进行交叉熵计算时与one-hot编码的差异

In [8]:
#标签总概率为1
labels = [[0.4,0.1,0.5],[0.3,0.6,0.1]]
result4 = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print ("rel4=",sess.run(result4),"\n") 

rel4= [2.1721554 2.7696736] 



从结果可以看出，非标准one-hot的label和logits之间的交叉熵之间的差异比标准one-hot小，说明非标准one-hot无法较好的表达分类正确和分类错误的交叉熵之间的差异

# 3. sparse交叉熵

sparse_softmax_cross_entropy_with_logits针对非标准one-hot编码设计，可以对非标准one-hot计算交叉熵，解决上面的问题

In [11]:
#sparse
labels = [2,1] #其实是0 1 2 三个类。等价 第一行 001 第二行 010，和原始标签相同
result5 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
with tf.Session() as sess:
    print ("result5=",sess.run(result5),"\n")

result5= [0.02215516 3.0996735 ] 



result5和result1相同，说明sparse交叉熵已经包含将非标准one-hot转化为标砖one-hot的操作

# 4. 计算loss

In [13]:
loss=tf.reduce_mean(result1)
with tf.Session() as sess:
    print ("loss=",sess.run(loss))
    
labels = [[0,0,1],[0,1,0]]    
# -tf.reduce_sum(labels * tf.log(logits_scaled),1)等价于softmax_cross_entropy_with_logits
loss2 = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(logits_scaled),1) )
with tf.Session() as sess:
    print ("loss2=",sess.run(loss2))

loss= 1.5609143
loss2= 1.5609144
